# Creating Brochure

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.
We will be provided a company name and their primary website.

Import Libraries

In [5]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

Configure llama connection via openai client python library

In [6]:
#Using the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
MODEL = "llama3.2"

In [16]:
# A class to represent a Webpage

# Some websites need to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [17]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have llm figure out which links are relevant

### Use a call to llama to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [38]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [19]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [20]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [21]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [22]:
def get_links(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

## Second step: make the brochure!

Assemble all the details into another prompt to llama 3.2

In [25]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [29]:
print(get_all_details("https://www.blrbrewing.com/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.blrbrewing.com'}, {'type': 'Company page', 'url': 'https://www.blrbrewing.com'}, {'type': 'Google Maps', 'url': 'https://maps.app.goo.gl/otMp1dLSzLRYNHAz5'}, {'type': 'Google Maps', 'url': 'https://maps.app.goo.gl/GipXrrdtbXsohWyx6'}, {'type': 'Google Maps', 'url': 'https://maps.app.goo.gl/uvFwXXD37QFYGYGs9'}, {'type': 'Company page', 'url': 'https://www.blrbrewing.com'}]}
Landing page:
Webpage Title:
BLR Brewing Co. - Electronic City | Kanakapura Road  | HSR Layout | Brewery | Restaurant
Webpage Contents:
top of page
HOME
ABOUT US
LOCATIONS
GALLERY
CONTACT
More
Use tab to navigate through the menu items.
BLR BREWING CO.
Curated with 5 Elements of Nature!
SERVICES
Our Dine-In Locations
Bellandur -
Eco World
They Bay at Eco world
​
+91-804568 2209
HSR Layout
Sector 4
+91-8068507771
Marathahalli - Prestige Tech Park
Gravity at Prestige Tech park
+91-804749 3555
Kanakapura Rd
Holiday Village rd
+91-804716 2222
Whitefield
​

In [30]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a mor  e humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [31]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [32]:
get_brochure_user_prompt("Bangalore Brewing", "https://www.blrbrewing.com/")

Found links: {'links': [{'type': 'Company page', 'url': 'https://www.blrbrewing.com'}, {'type': 'Facebook page', 'url': 'http://www.facebook.com/blrbrewing'}, {'type': 'Instagram page', 'url': 'https://instagram.com/blrbrewing'}, {'type': 'Google Maps location page', 'url': 'https://maps.app.goo.gl/kp7xcws96EkCdeS3A'}, {'type': 'Google Maps location page', 'url': 'https://maps.app.goo.gl/zYCwKb7gL9FwcqXJ7'}, {'type': 'Google Maps location page', 'url': 'https://maps.app.goo.gl/otMp1dLSzLRYNHAz5'}, {'type': 'Google search result for Blr Brewing in Kanakapura Road', 'url': 'https://www.google.com/search?q=blr+brewing+kanakapura+road&rlz=1C5CHFA_enIN740IN741&oq=blr+brewing+kan&aqs=chrome.0.35i39j46i175i199i512j69i57j0i22i30j69i60l3.3246j0j7&sourceid=chrome&ie=UTF-8#'}, {'type': 'Google search result for Blr Brewing in Electronic City', 'url': 'https://www.google.com/search?q=blr+brewing+electronic+city&rlz=1C5CHFA_enIN740IN741&oq=blr+&aqs=chrome.0.69i59j69i57j69i60l2j69i61j69i60j69i65l2.9

"You are looking at a company called: Bangalore Brewing\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nBLR Brewing Co. - Electronic City | Kanakapura Road  | HSR Layout | Brewery | Restaurant\nWebpage Contents:\ntop of page\nHOME\nABOUT US\nLOCATIONS\nGALLERY\nCONTACT\nMore\nUse tab to navigate through the menu items.\nBLR BREWING CO.\nCurated with 5 Elements of Nature!\nSERVICES\nOur Dine-In Locations\nBellandur -\nEco World\nThey Bay at Eco world\n\u200b\n+91-804568 2209\nHSR Layout\nSector 4\n+91-8068507771\nMarathahalli - Prestige Tech Park\nGravity at Prestige Tech park\n+91-804749 3555\nKanakapura Rd\nHoliday Village rd\n+91-804716 2222\nWhitefield\n\u200b\nSeetharamapalya Metro Stn.\n+91-8071174545\nElectronic City\nNeeladri Rd\n+91-80\n4710 8277\nOUR WORK\nGallery\nGlimpse of our food & drink curated by our head chef & mixologist\nABOUT US\nAbout BLR 

In [33]:
def create_brochure(company_name, url):
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [34]:
create_brochure("Bangalore Brewing", "https://www.blrbrewing.com/")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.blrbrewing.com'}, {'type': 'facebook page', 'url': 'http://www.facebook.com/blrbrewing'}, {'type': 'instagram profile', 'url': 'https://instagram.com/blrbrewing'}]}


# Welcome to BLR Brewing Co.

## About Us

BLR Brewing Co. is Bengaluru's leading brewpub chain, bringing a unique blend of nature and urban charm to the city. Our four distinct locations in Electronic City, Kanakapura Road, HSR Layout, and Whitefield are curated with five elements of nature - Earth, Water, Air, Fire, and Space.

Our story begins with a passion for brewing and a desire to create a unique dining experience. We embrace the essence of Bangalore's rich brewing tradition while bringing modern flavors and refreshing brews to our menu.

## Our Venues

* **Electronic City**: 84/4, 4th Floor, Neeladri Rd, opposite to Wipro Gate
* **Kanakapura Road**: Paramount Gardens, Jyotipuram, Bengaluru, Karnataka 560062
* **HSR Layout**: 450, 1st Floor, 17th Cross Rd, Sector 4
* **Whitefield**: No. 2, behind HDFC Bank, Doddanakundi Industrial Area 2

## Services

* **Dine-In Locations**: Experience our globally inspired menu with live gigs, DJ nights, and indulgent Sunday brunches at any of our four locations.
* **Reservations**: Book your table now and enjoy a memorable dining experience!

## Contact Us

Phone Numbers:

* Electronic City: +91-8047162222
* Kanakapura Road: +91-8047108277
* HSR Layout: +91-8068507771
* Whitefield: +91-8071174545
* Marathahalli: None listed (may be opening soon)

## Gallery

View our food and drink curated by our head chef & mixologist:

[Insert gallery link]

Join us for an unforgettable experience amidst Bangalore's rich brewing tradition. Explore our menu, book a table, and get in touch with us today!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [35]:
def stream_brochure(company_name, url):
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [36]:
stream_brochure("Bangalore Brewing", "https://www.blrbrewing.com/")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.blrbrewing.com'}, {'type': 'facebook page', 'url': 'http://www.facebook.com/blrbrewing'}, {'type': 'instagram page', 'url': 'https://instagram.com/blrbrewing'}, {'type': 'google maps', 'url': 'https://maps.app.goo.gl/kp7xcws96EkCdeS3A'}, {'type': 'google maps 2nd instance', 'url': 'https://maps.app.goo.gl/zYCwKb7gL9FwcqXJ7'}]}


# Bangalore Brewing Co.
## A Harmonious Blend of Nature and Urban Charm

[Image of BLR Brewing Co's logo]

About Us
--------

At BLR Brewing Co., we stand as Bengaluru's leading brewpub chain, spread across four distinct locales: Electronic City, Kanakapura Road, HSR Layout, and Whitefield. Our journey is inspired by the five elements - Earth, Water, Air, Fire, and Space - which we've curated to create a unique dining experience.

Our Vibe
---------

*   Lush greenery, tranquil water bodies, and vintage Bangalore décor come together to create an inviting atmosphere.
*   Live gigs, DJ nights, and indulgent Sunday brunches promise an unforgettable experience for patrons of all ages and backgrounds.

Our Cuisine
------------

*   Our globally inspired menu offers mouthwatering delights and refreshing brews.
*   Dine in our four distinct locations: Bellandur - Eco World, HSR Layout, Marathahalli - Prestige Tech Park, and Whitefield.

**Locations**
We have the following dine-in locations:

1.  **Bellandur - Eco World:** [84/4, 4th Floor, Neeladri Rd, opposite to Wipro Gate](tel:+91-8047162222)
2.  **HSR Layout:** [450, 1st Floor, 17th Cross Rd](tel:+91-8068507771)
3.  **Marathahalli - Prestige Tech Park:** [Gravity at Prestige Tech Park](tel: +91-8047493555)
4.  **Whitefield:** [No. 2, behind HDFC Bank](tel:+91-8071174545)

**Opening Hours**
MON - TUE: 12:00 PM - 11:30 PM
WED - SUN: 12:00 PM -01:00 AM

[Image of BLR Brewing Co's menu]

At BLR Brewing Co., we're passionate about brewing and serving exceptional food. Explore our globally inspired menu.

## Careers
*   We are always looking for new talent to join our team.
*   Check out our latest job openings in various areas such as brewery operations, kitchen management, and customer service.

[Image of BLR Brewing Co's logo]

**Contact Us**
Phone: +91-80 4710 8277
Email: [info@blrbrewingco.com](mailto:info@blrbrewingco.com)

Stay updated with the latest news and events about BLR Brewing Co. by following us on social media.

In [37]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Bangalore Brewing", "https://www.blrbrewing.com/")

Found links: {'links': [{'type': 'About page', 'url': 'https://www.blrbrewing.com'}, {'type': 'Company page', 'url': 'https://www.blrbrewing.com'}, {'type': 'Google Maps link', 'url': 'https://maps.app.goo.gl/uvFwXXD37QFYGYGs9'}]}


**Welcome to BLR Brewing Co.**
==========================

[Cover Image: A serene and vibrant photo of one of our brewery locations]

**Embracing the Essence of Nature**

BLR Brewing Co. is Bengaluru's leading brewpub chain, dedicated to creating a harmonious blend of nature and urban charm. Our four distinct venues in Electronic City, Kanakapura Road, HSR Layout, and Whitefield embody the five elements of nature: Earth, Water, Air, Fire, and Space.

**Our Story**

Our mission is to bring people together through great food, refreshing brews, and lively entertainment. We pride ourselves on using only the finest ingredients and traditional brewing methods to craft our unique beers and menus. Our head chef and mixologist carefully curate each dish to transport your taste buds on a culinary journey.

**Our Venues**

* **Electronic City**: Our flagship brewery in Electronics City Phase 1, featuring an expansive lawn area perfect for picnics and outdoor gatherings.
* **Kanakapura Road**: Situated in Paramount Gardens, Jyotipuram, our Kanakapura Road location boasts a tranquil ambiance with lush greenery and water features.
* **HSR Layout**: Conveniently located near Sector 4, HSR Layout, our venue offers stunning views of the city while enjoying live gigs and DJ nights.
* **Whitefield**: Located behind HDFC Bank in Seetharamapalya, Hoodi, our Whitefield location is an ideal spot for outdoor entertainment and a relaxing atmosphere.

**Our Values**

At BLR Brewing Co., we value:

*   Quality ingredients and traditional brewing methods
*   Excellent customer service and personalized attention
*   Sustainable practices that minimize our impact on the environment
*   Collaborative relationships with local artists, musicians, and businesses

**Join Us**

Whether you're a foodie, beer enthusiast, or just looking for a great time with friends and family, BLR Brewing Co. has something for everyone. Reserve your table now and experience the best of Bangalore's brewing tradition.

[Contact Information]

*   Phone: +91-80 4710 8277
*   Email: [info@blrbrewing.com](mailto:info@blrbrewing.com)
*   Find Us:
    <ul>
        *   Electronic City: 84/4, 4th Floor, Neeladri Rd, opposite to Wipro Gate, Karuna Nagar, Electronics City Phase 1, Bengaluru, Karnataka 560100
        *   Kanakapura Road: Paramount Gardens, Jyotipuram, Bengaluru, Karnataka 560062
        *   HSR Layout: 450, 1st Floor, 17th Cross Rd, Sector 4, HSR Layout, Bengaluru, Karnataka 560102
        *   Whitefield: No. 2, behind HDFC Bank, Doddanakundi Industrial Area 2, Seetharamapalya, Hoodi, Bengaluru, Karnataka 560048

**Stay Connected**

Follow us on social media for the latest updates, promotions, and events:

[Social Media Links]

*   Facebook: @blrbrewingco
*   Instagram: @blrbrewingco
*   Twitter: @blrbrewingco